In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master('local[*]') \
    .appName("USA Accidents Analysis with Pyspark") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/03 16:07:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark

Looks like we initialized it successfully
Now to reading the data into a spark DF

In [3]:
main_df = spark.read.csv('./Data/US_Accidents_Dec21_updated.csv')
main_df.columns

['_c0',
 '_c1',
 '_c2',
 '_c3',
 '_c4',
 '_c5',
 '_c6',
 '_c7',
 '_c8',
 '_c9',
 '_c10',
 '_c11',
 '_c12',
 '_c13',
 '_c14',
 '_c15',
 '_c16',
 '_c17',
 '_c18',
 '_c19',
 '_c20',
 '_c21',
 '_c22',
 '_c23',
 '_c24',
 '_c25',
 '_c26',
 '_c27',
 '_c28',
 '_c29',
 '_c30',
 '_c31',
 '_c32',
 '_c33',
 '_c34',
 '_c35',
 '_c36',
 '_c37',
 '_c38',
 '_c39',
 '_c40',
 '_c41',
 '_c42',
 '_c43',
 '_c44',
 '_c45',
 '_c46']

Not quite what we were looking for, let's fetch the headers too and prettify the print with the schema.

In [4]:
main_df = spark.read.csv('./Data/US_Accidents_Dec21_updated.csv', header=True)
main_df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Severity: string (nullable = true)
 |-- Start_Time: string (nullable = true)
 |-- End_Time: string (nullable = true)
 |-- Start_Lat: string (nullable = true)
 |-- Start_Lng: string (nullable = true)
 |-- End_Lat: string (nullable = true)
 |-- End_Lng: string (nullable = true)
 |-- Distance(mi): string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Number: string (nullable = true)
 |-- Street: string (nullable = true)
 |-- Side: string (nullable = true)
 |-- City: string (nullable = true)
 |-- County: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Zipcode: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Timezone: string (nullable = true)
 |-- Airport_Code: string (nullable = true)
 |-- Weather_Timestamp: string (nullable = true)
 |-- Temperature(F): string (nullable = true)
 |-- Wind_Chill(F): string (nullable = true)
 |-- Humidity(%): string (nullable = true)
 |-- Pressure(in): strin

While that worked, Spark inferred that all the columns are strings, even the ID!
Let's take a look


In [5]:
main_df.select('ID').show(20)

+----+
|  ID|
+----+
| A-1|
| A-2|
| A-3|
| A-4|
| A-5|
| A-6|
| A-7|
| A-8|
| A-9|
|A-10|
|A-11|
|A-12|
|A-13|
|A-14|
|A-15|
|A-16|
|A-17|
|A-18|
|A-19|
|A-20|
+----+
only showing top 20 rows



That's a weird choice for an id, we will have to clean that.
Let's find some other columns that seem ... suspicious.
Using a select with a show method will return columns' names which will make it unreadable. Let's try collect ...

In [6]:
main_df.select('*').limit(1).collect()

22/08/03 16:07:45 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


[Row(ID='A-1', Severity='3', Start_Time='2016-02-08 00:37:08', End_Time='2016-02-08 06:37:08', Start_Lat='40.108909999999995', Start_Lng='-83.09286', End_Lat='40.11206', End_Lng='-83.03187', Distance(mi)='3.23', Description='Between Sawmill Rd/Exit 20 and OH-315/Olentangy Riv Rd/Exit 22 - Accident.', Number=None, Street='Outerbelt E', Side='R', City='Dublin', County='Franklin', State='OH', Zipcode='43017', Country='US', Timezone='US/Eastern', Airport_Code='KOSU', Weather_Timestamp='2016-02-08 00:53:00', Temperature(F)='42.1', Wind_Chill(F)='36.1', Humidity(%)='58.0', Pressure(in)='29.76', Visibility(mi)='10.0', Wind_Direction='SW', Wind_Speed(mph)='10.4', Precipitation(in)='0.0', Weather_Condition='Light Rain', Amenity='False', Bump='False', Crossing='False', Give_Way='False', Junction='False', No_Exit='False', Railway='False', Roundabout='False', Station='False', Stop='False', Traffic_Calming='False', Traffic_Signal='False', Turning_Loop='False', Sunrise_Sunset='Night', Civil_Twilight

That's more readable!
Timestamps look ok, maybe we will separate dates from times later, but for now this works.
It seems like many columns are Boolean type and the last 4 columns take discrete values
Let's see what these values are

We already know that the severity column takes values from 1-4 from the kaggle description.

In [7]:
main_df.select('Sunrise_Sunset','Civil_Twilight','Nautical_Twilight','Astronomical_Twilight').distinct().show()

+--------------+--------------+-----------------+---------------------+
|Sunrise_Sunset|Civil_Twilight|Nautical_Twilight|Astronomical_Twilight|
+--------------+--------------+-----------------+---------------------+
|         Night|         Night|            Night|                  Day|
|          null|          null|             null|                 null|
|           Day|           Day|              Day|                  Day|
|           Day|         Night|              Day|                  Day|
|         Night|         Night|              Day|                  Day|
|         Night|           Day|              Day|                  Day|
|         Night|           Day|              Day|                Night|
|           Day|         Night|              Day|                Night|
|         Night|         Night|              Day|                Night|
|           Day|           Day|              Day|                Night|
|         Night|         Night|            Night|               

While this obviously takes the distinct of the combination of all four columns together,
not the distinct value of each column, it fulfills our purpose for now.
An alternate way was to union each query separately and then call distinct on them
unnecessary for this case. 

Some columns are clearly Boolean. Will need to verify that when loading the data to avoid any surprises.
Let's look at other columns for now.

In [21]:
from pyspark.sql.functions import lit
main_df.select(lit(1)).where((main_df.Start_Lng.isNull()) | 
                      (main_df.Start_Lat.isNull()) |
                      (main_df.End_Lng.isNull()) |
                       (main_df.End_Lat.isNull())).show()

+---+
|  1|
+---+
+---+



In [22]:
main_df.select('Amenity','Bump','Crossing','Give_Way','Junction','No_Exit','Railway','Roundabout'
                ,'Station','Stop','Traffic_Calming','Traffic_Signal','Turning_Loop').distinct().show()

+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+------------+
|Amenity| Bump|Crossing|Give_Way|Junction|No_Exit|Railway|Roundabout|Station| Stop|Traffic_Calming|Traffic_Signal|Turning_Loop|
+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+------------+
|  False|False|    True|   False|    True|  False|  False|     False|  False|False|          False|          True|       False|
|   True|False|   False|   False|   False|   True|  False|     False|  False|False|          False|          True|       False|
|  False|False|   False|   False|    True|  False|   True|     False|  False|False|          False|         False|       False|
|  False|False|   False|   False|   False|  False|  False|     False|  False|False|          False|         False|       False|
|   True|False|   False|   False|   False|   True|  False|     False|   True|False|          False|     

So the boolean columns and the coordinates have no nulls.

In [ ]:
main_df.select('Weather_condition').distinct().show()

Wow, I wasn't expecting that. Let's see how many unique values there are.

In [ ]:
main_df.select('Weather_condition').distinct().count()

That's alot, hm ....
This will need some work.

From the dataset description on Kaggle, we know that severity is between 1 and 4, we know that all the data should be in the US. We can try to verify that by checking that the coordinates given lie inside the coordinates of the US.

One last step. We need to know the first start date and the last end date. You will see why in the next script.

#main_df.agg({"Start_Time":"min"}).show()
#main_df.agg({"End_Time":"max"}).show()
We can use these two above, but I prefer running a SQL query for this one instead.

In [ ]:
main_df.createOrReplaceTempView('main_df')
spark.sql('SELECT MIN(Start_Time),MAX(End_Time) FROM main_df').show()

Huh, that end date looks ... odd, is this a default value?

In [ ]:
main_df.select('*').filter("End_Time = '2022-01-01 00:00:00'").collect()

No way that road was closed for almost 6 months! Let's use the second latest End time

In [ ]:
main_df.select('Start_Time','End_Time').orderBy('End_time', ascending = False).limit(5).show()

The first two seem weird. It's ok, we will just use their end times as nulls.